In [1]:
import pandas as pd #type: ignore
import numpy as np

# Load the ordered data
df = pd.read_csv('processed_data/ordered_times.csv')

# Define the order lists (same as in data_wrangler.ipynb)
sheet_order = [
    #'CMS All Time Top 10',
    #'CMS Axelrood Pool Records',
    #'CMS Frosh Swimming & Diving Records',
    #'Development of Team Records (October 2001 to March 2025)', 
    'CMS at UCSD',
    'CMS at Cal Baptist Distance Meet',
    'CMS at PP',
    'CMS at PP Combined', 
    'CMS SCIAC Champions',
    'SCIAC All Time Top 10 Performers',
    'SCIAC Records',
    'NCAA TOP 20'
]

sex_order = ['Athena', 'Stag', 'Women', 'Men']

event_order = [
    # FREE
    '50 FREE', '100 FREE', '200 FREE', '500 FREE', '1000 FREE', '1650 FREE',
    # BACK
    '50 BACK', '100 BACK', '200 BACK', '300 BACK',
    # BREAST
    '50 BREAST', '100 BREAST', '200 BREAST', '300 BREAST',
    # FLY
    '100 FLY', '200 FLY', '300 FLY',
    # IM
    '200 IM', '300 IM', '400 IM',
    # DIVING (METER)
    '1-METER (6 dives)', '1-METER (11 dives)', '1-METER',
    '3-METER (6 dives)', '3-METER (11 dives)', '3-METER',
    # RELAY
    '200 FREE RELAY', '400 FREE RELAY', '500 FREE RELAY- (50-100-150-200)',
    '800 FREE RELAY', '200 MEDLEY RELAY', '400 MEDLEY RELAY',
    '500 MEDLEY RELAY - (200 BACK-150 BR-100 FL-50 FS)',
    # Spl.
    '50 FREE - RELAY Spl.', '50 FREE Spl.', '100 FREE - RELAY Spl.',
    '100 FREE Spl.', '200 FREE - RELAY Spl.', '200 FREE Spl.',
    '50 BACK - RELAY Spl.', '50 BACK Spl.', '50 BREAST - RELAY Spl.',
    '50 BREAST Spl.', '100 BREAST - RELAY Spl.', '100 BREAST Spl.',
    '50 FLY - RELAY Spl.', '50 FLY Spl.', '100 FLY - RELAY Spl.', '100 FLY Spl.'
]

print(f"Loaded {len(df)} records")
print(f"Unique sheets: {df['SHEET'].nunique()}")
print(f"Unique events: {df['EVENT'].nunique()}")


Loaded 4173 records
Unique sheets: 12
Unique events: 49


In [35]:
class SimpleSwimTableGenerator:
    def __init__(self, df, sheet_order, sex_order, event_order):
        self.df = df
        self.sheet_order = sheet_order
        self.sex_order = sex_order
        self.event_order = event_order
        
        # Override system for special cases
        self.overrides = {
            'RELAY': {'layout': 'single_column', 'max_entries': None},
            'Spl.': {'layout': 'single_column', 'max_entries': None},
            'METER': {'layout': 'single_column', 'max_entries': None},
            'CMS All Time Top 10': {'max_entries': 10},
            'CMS at PP': {'max_entries': 10}
        }
    
    def time_to_seconds(self, time_str):
        """Convert time string to seconds for sorting"""
        if pd.isna(time_str):
            return float('inf')
        try:
            time_str = str(time_str).strip()
            if ':' in time_str:
                parts = time_str.split(':')
                minutes = float(parts[0])
                seconds = float(parts[1])
                return minutes * 60 + seconds
            else:
                return float(time_str)
        except (ValueError, AttributeError):
            return float('inf')
    
    def get_table_columns(self, data):
        """Determine which columns to include in the table"""
        available_cols = []
        
        # Clean column names by stripping whitespace
        data_columns = [col.strip() for col in data.columns]
        data.columns = data_columns
        
        # Check each column and include it if it has data
        for col in ['TIME', 'NAME', 'YEAR', 'TEAM', 'RANK', 'SITE', 'MEET', 'CONTEXT', 'DATE']:
            if col in data.columns:
                # Check if column has any non-null, non-empty values
                non_null_count = data[col].notna().sum()
                non_empty_count = (data[col].astype(str).str.strip() != '').sum()
                
                if non_null_count > 0 and non_empty_count > 0:
                    available_cols.append(col)
        
        return available_cols
    
    def get_column_widths(self, columns):
        """Get appropriate column widths based on column types - optimized for wider tables"""
        width_map = {
            'TIME': '1.8cm',
            'NAME': '2.8cm', 
            'YEAR': '1.2cm',
            'TEAM': '1.4cm',
            'RANK': '0.8cm',
            'SITE': '1.4cm',
            'MEET': '1.4cm',
            'CONTEXT': '2.0cm',
            'DATE': '1.2cm'
        }
        return [width_map.get(col, '1.5cm') for col in columns]
    
    def get_column_headers(self, columns):
        """Get LaTeX headers for columns"""
        header_map = {
            'TIME': '\\textbf{TIME}',
            'NAME': '\\textbf{NAME}',
            'YEAR': '\\textbf{YEAR}',
            'TEAM': '\\textbf{TEAM}',
            'RANK': '\\textbf{RANK}',
            'SITE': '\\textbf{SITE}',
            'MEET': '\\textbf{MEET}',
            'CONTEXT': '\\textbf{CONTEXT}',
            'DATE': '\\textbf{DATE}'
        }
        return [header_map.get(col, f'\\textbf{{{col}}}') for col in columns]
    
    def estimate_table_height(self, num_entries, num_columns):
        """Estimate table height in lines for page planning"""
        # Base height: title (1) + header (1) + bottom border (1) = 3 lines
        # Each data row = 1 line
        # Add some padding
        return 3 + num_entries + 1
    
    def should_use_single_column(self, event_name, num_entries):
        """Determine if event should use single column layout"""
        # Check overrides first
        for key, config in self.overrides.items():
            if key in event_name:
                if 'layout' in config and config['layout'] == 'single_column':
                    return True
        
        # Default logic - be more conservative about single column
        if 'RELAY' in event_name or 'Spl.' in event_name or 'METER' in event_name:
            return True
        if num_entries > 20:  # Increased threshold
            return True
        return False
    
    def group_events_for_layout(self, events_data):
        """Group events into efficient grid layout - 2 columns x 3 rows per page"""
        groups = []
        i = 0
        
        while i < len(events_data):
            # Create a row with up to 2 events
            row_events = []
            
            # First event in row
            if i < len(events_data):
                event_name, event_data = events_data[i]
                row_events.append((event_name, event_data))
                i += 1
            
            # Second event in row (if available and not single column)
            if i < len(events_data):
                next_event_name, next_event_data = events_data[i]
                if not self.should_use_single_column(next_event_name, len(next_event_data)):
                    row_events.append((next_event_name, next_event_data))
                    i += 1
            
            groups.append(row_events)
        
        return groups
    
    def generate_table_latex(self, event_name, data):
        """Generate LaTeX for a single table using direct tabular environment"""
        # Sort by time if TIME column exists
        if 'TIME' in data.columns:
            data = data.copy()
            data['time_seconds'] = data['TIME'].apply(self.time_to_seconds)
            data = data.sort_values('time_seconds').drop('time_seconds', axis=1)
            data = data.reset_index(drop=True)
        
        # Apply max entries override
        max_entries = None
        for key, config in self.overrides.items():
            if key in event_name or key in data['SHEET'].iloc[0] if len(data) > 0 else False:
                if 'max_entries' in config:
                    max_entries = config['max_entries']
                    break
        
        if max_entries and len(data) > max_entries:
            data = data.head(max_entries)
        
        # Get columns to include
        columns = self.get_table_columns(data)
        if not columns:
            return f"\\textbf{{{event_name}}}\\\\[0.1cm]\\textit{{No data available}}"
        
        # Get column widths and headers
        widths = self.get_column_widths(columns)
        headers = self.get_column_headers(columns)
        
        # Build tabular specification
        col_spec = '@{}' + 'p{' + '}p{'.join(widths) + '}@{}'
        
        # Generate table rows with better formatting for manual editing
        rows = []
        for _, row in data.iterrows():
            row_data = []
            for col in columns:
                value = str(row[col]) if pd.notna(row[col]) else ""
                # Escape special LaTeX characters
                value = value.replace('&', '\\&').replace('%', '\\%').replace('$', '\\$')
                row_data.append(value)
            rows.append("    " + " & ".join(row_data) + " \\\\")
        
        table_content = "\n".join(rows)
        header_row = "    " + " & ".join(headers) + " \\\\"
        
        # Build complete table with clear structure for manual editing
        table_latex = f"""\\centering
\\textbf{{{event_name}}}\\\\[0.1cm]
\\begin{{tabular}}{{{col_spec}}}
\\hline
{header_row}
\\hline
{table_content}
\\hline
\\end{{tabular}}"""
        
        return table_latex
    
    def generate_section_latex(self, sheet_name, sex_name, events_data):
        """Generate complete LaTeX section optimized for manual editing"""
        latex_parts = []
        
        # Escape ampersands in section titles
        escaped_sheet_name = sheet_name.replace('&', '\\&')
        escaped_sex_name = sex_name.replace('&', '\\&')
        
        # Add clear section markers for manual editing
        latex_parts.append(f"% ===== {escaped_sheet_name} - {escaped_sex_name} =====")
        latex_parts.append(f"\\subsection{{{escaped_sheet_name}}}")
        latex_parts.append(f"\\subsubsection{{{escaped_sex_name}}}")
        latex_parts.append("")
        
        # Group events for layout
        event_groups = self.group_events_for_layout(events_data)
        
        for i, group in enumerate(event_groups):
            # Add table group comment for easy identification
            if len(group) == 2:
                event1_name, event2_name = group[0][0], group[1][0]
                latex_parts.append(f"% Table Group {i+1}: {event1_name} + {event2_name}")
            else:
                event_name = group[0][0]
                latex_parts.append(f"% Table Group {i+1}: {event_name}")
            
            if len(group) == 2:
                # Two-column layout - optimized for page usage
                event1_name, event1_data = group[0]
                event2_name, event2_data = group[1]
                
                table1 = self.generate_table_latex(event1_name, event1_data)
                table2 = self.generate_table_latex(event2_name, event2_data)
                
                latex_parts.append("\\begin{table}[H]")
                latex_parts.append("\\centering")
                latex_parts.append("\\begin{minipage}[t]{0.48\\textwidth}")
                latex_parts.append(table1)
                latex_parts.append("\\end{minipage}\\hfill")
                latex_parts.append("\\begin{minipage}[t]{0.48\\textwidth}")
                latex_parts.append(table2)
                latex_parts.append("\\end{minipage}")
                latex_parts.append("\\end{table}")
                latex_parts.append("")
            else:
                # Single column layout - optimized width
                event_name, event_data = group[0]
                table = self.generate_table_latex(event_name, event_data)
                
                latex_parts.append("\\begin{table}[H]")
                latex_parts.append("\\centering")
                latex_parts.append("\\begin{minipage}[t]{0.6\\textwidth}")
                latex_parts.append(table)
                latex_parts.append("\\end{minipage}")
                latex_parts.append("\\end{table}")
                latex_parts.append("")
        
        # Add section end marker
        latex_parts.append(f"% ===== END {escaped_sheet_name} - {escaped_sex_name} =====")
        latex_parts.append("")
        
        return "\n".join(latex_parts)

# Create the generator
generator = SimpleSwimTableGenerator(df, sheet_order, sex_order, event_order)
print("Simple generator created!")


Simple generator created!


In [36]:
# Generate complete LaTeX for all sections
def generate_complete_latex():
    """Generate LaTeX for all sheet/sex combinations with manual editing support"""
    all_latex = []
    
    # Add file header with summary information
    all_latex.append("% ===== CMS MEDIA GUIDE - GENERATED LATEX =====")
    all_latex.append("% This file contains all swimming and diving tables")
    all_latex.append("% Optimized for manual editing and page layout")
    all_latex.append("")
    
    # Generate summary of sections
    all_latex.append("% ===== SECTION SUMMARY =====")
    section_count = 0
    for sheet in sheet_order:
        sheet_data = df[df['SHEET'] == sheet]
        if len(sheet_data) == 0:
            continue
            
        available_sexes = sheet_data['SEX'].unique()
        for sex in sex_order:
            if sex in available_sexes:
                section_count += 1
                all_latex.append(f"% {section_count}. {sheet} - {sex}")
    
    all_latex.append(f"% Total sections: {section_count}")
    all_latex.append("")
    all_latex.append("% ===== END SUMMARY =====")
    all_latex.append("")
    
    # Generate actual content
    for sheet in sheet_order:
        sheet_data = df[df['SHEET'] == sheet]
        if len(sheet_data) == 0:
            continue
            
        # Get available sexes for this sheet
        available_sexes = sheet_data['SEX'].unique()
        
        for sex in sex_order:
            if sex not in available_sexes:
                continue
                
            sex_data = sheet_data[sheet_data['SEX'] == sex]
            
            # Group by event in order
            events_data = []
            for event in event_order:
                event_data = sex_data[sex_data['EVENT'] == event]
                if len(event_data) > 0:
                    events_data.append((event, event_data))
            
            if events_data:  # Only generate if there's data
                section_latex = generator.generate_section_latex(sheet, sex, events_data)
                all_latex.append(section_latex)
                
                # Add page break after each section
                all_latex.append("\\newpage")
                all_latex.append("")
    
    return "\n".join(all_latex)

# Generate the complete LaTeX
print("Generating complete LaTeX...")
complete_latex = generate_complete_latex()

# Save to file
with open('/home/ben/Desktop/Projects/media_guide/latex/sections/generated_latex.tex', 'w') as f:
    f.write(complete_latex)

print(f"Generated LaTeX saved to 'generated_latex.tex'")
print(f"Total length: {len(complete_latex)} characters")
print(f"Number of lines: {complete_latex.count(chr(10))}")

# Show preview
print("\nPreview (first 15 lines):")
lines = complete_latex.split('\n')
for i, line in enumerate(lines[:15]):
    print(f"{i+1:2d}: {line}")
if len(lines) > 15:
    print(f"... and {len(lines) - 15} more lines")

# Check for the problematic section
print("\nChecking for Development of Team Records section:")
dev_section_lines = [i for i, line in enumerate(lines) if "Development of Team Records" in line]
for line_num in dev_section_lines:
    print(f"Line {line_num+1}: {lines[line_num]}")
    # Show a few lines after this
    for j in range(1, 6):
        if line_num + j < len(lines):
            print(f"Line {line_num+j+1}: {lines[line_num + j]}")


Generating complete LaTeX...
Generated LaTeX saved to 'generated_latex.tex'
Total length: 333984 characters
Number of lines: 10550

Preview (first 15 lines):
 1: % ===== CMS MEDIA GUIDE - GENERATED LATEX =====
 2: % This file contains all swimming and diving tables
 3: % Optimized for manual editing and page layout
 4: 
 5: % ===== SECTION SUMMARY =====
 6: % 1. CMS All Time Top 10 - Athena
 7: % 2. CMS All Time Top 10 - Stag
 8: % 3. CMS Axelrood Pool Records - Women
 9: % 4. CMS Axelrood Pool Records - Men
10: % 5. CMS Frosh Swimming & Diving Records - Athena
11: % 6. CMS Frosh Swimming & Diving Records - Stag
12: % 7. Development of Team Records (October 2001 to March 2025) - Athena
13: % 8. Development of Team Records (October 2001 to March 2025) - Stag
14: % 9. CMS at UCSD - Athena
15: % 10. CMS at UCSD - Stag
... and 10536 more lines

Checking for Development of Team Records section:
Line 12: % 7. Development of Team Records (October 2001 to March 2025) - Athena
Line 13: % 8. Dev

In [2]:
# Generate dual meet tables using teamrecordtable macro
def generate_dual_meet_tables():
    """Generate LaTeX for dual meet sheets using teamrecordtable macro in 2-column layout"""
    
    # Define the dual meet sheets we want to process
    dual_meet_sheets = [
        'CMS at UCSD',
        'CMS at Cal Baptist Distance Meet', 
        'CMS at PP'
    ]
    
    # Filter data for these sheets only
    dual_data = df[df['SHEET'].isin(dual_meet_sheets)]
    
    print(f"Found {len(dual_data)} records for dual meets")
    print(f"Sheets: {dual_meet_sheets}")
    
    latex_parts = []
    
    # Add header
    latex_parts.append("% ===== DUAL MEET TABLES =====")
    latex_parts.append("% Generated using teamrecordtable macro")
    latex_parts.append("")
    
    # Process each sheet
    for sheet in dual_meet_sheets:
        sheet_data = dual_data[dual_data['SHEET'] == sheet]
        if len(sheet_data) == 0:
            continue
            
        print(f"\nProcessing {sheet}...")
        
        # Escape sheet name for LaTeX
        escaped_sheet_name = sheet.replace('&', '\\&')
        latex_parts.append(f"\\subsection{{{escaped_sheet_name}}}")
        latex_parts.append("")
        
        # Get available sexes for this sheet
        available_sexes = sheet_data['SEX'].unique()
        
        for sex in ['Athena', 'Stag', 'Women', 'Men']:
            if sex not in available_sexes:
                continue
                
            sex_data = sheet_data[sheet_data['SEX'] == sex]
            
            # Escape sex name for LaTeX
            escaped_sex_name = sex.replace('&', '\\&')
            latex_parts.append(f"\\subsubsection{{{escaped_sex_name}}}")
            latex_parts.append("")
            
            # Group by event in order
            events_data = []
            for event in event_order:
                event_data = sex_data[sex_data['EVENT'] == event]
                if len(event_data) > 0:
                    # Sort by time if TIME column exists
                    if 'TIME' in event_data.columns:
                        event_data = event_data.copy()
                        event_data['time_seconds'] = event_data['TIME'].apply(
                            lambda x: float('inf') if pd.isna(x) else 
                            (float(x.split(':')[0]) * 60 + float(x.split(':')[1]) if ':' in str(x) else float(x))
                        )
                        event_data = event_data.sort_values('time_seconds').drop('time_seconds', axis=1)
                        event_data = event_data.reset_index(drop=True)
                    events_data.append((event, event_data))
            
            # Group events in pairs for 2-column layout
            for i in range(0, len(events_data), 2):
                latex_parts.append("\\begin{table}[H]")
                latex_parts.append("\\centering")
                
                # First event
                if i < len(events_data):
                    event_name, event_data = events_data[i]
                    
                    # Generate table rows for teamrecordtable macro
                    rows = []
                    for _, row in event_data.iterrows():
                        time_val = str(row['TIME']) if pd.notna(row['TIME']) else ""
                        name_val = str(row['NAME']) if pd.notna(row['NAME']) else ""
                        year_val = str(row['YEAR']) if pd.notna(row['YEAR']) else ""
                        
                        # Escape special LaTeX characters
                        time_val = time_val.replace('&', '\\&').replace('%', '\\%')
                        name_val = name_val.replace('&', '\\&').replace('%', '\\%')
                        
                        rows.append(f"    {time_val} & {name_val} & {year_val} \\\\")
                    
                    table_content = "\n".join(rows)
                    
                    latex_parts.append("\\begin{minipage}[t]{0.48\\textwidth}")
                    latex_parts.append(f"\\teamrecordtable{{{event_name}}}{{")
                    latex_parts.append(table_content)
                    latex_parts.append("}")
                    latex_parts.append("\\end{minipage}\\hfill")
                
                # Second event (if available)
                if i + 1 < len(events_data):
                    event_name, event_data = events_data[i + 1]
                    
                    # Generate table rows for teamrecordtable macro
                    rows = []
                    for _, row in event_data.iterrows():
                        time_val = str(row['TIME']) if pd.notna(row['TIME']) else ""
                        name_val = str(row['NAME']) if pd.notna(row['NAME']) else ""
                        year_val = str(row['YEAR']) if pd.notna(row['YEAR']) else ""
                        
                        # Escape special LaTeX characters
                        time_val = time_val.replace('&', '\\&').replace('%', '\\%')
                        name_val = name_val.replace('&', '\\&').replace('%', '\\%')
                        
                        rows.append(f"    {time_val} & {name_val} & {year_val} \\\\")
                    
                    table_content = "\n".join(rows)
                    
                    latex_parts.append("\\begin{minipage}[t]{0.48\\textwidth}")
                    latex_parts.append(f"\\teamrecordtable{{{event_name}}}{{")
                    latex_parts.append(table_content)
                    latex_parts.append("}")
                    latex_parts.append("\\end{minipage}")
                else:
                    # If only one event, close the minipage properly
                    latex_parts[-1] = "\\end{minipage}"  # Remove \hfill
                
                latex_parts.append("\\end{table}")
                latex_parts.append("")
    
    return "\n".join(latex_parts)

# Generate the dual meet tables
print("Generating dual meet tables...")
dual_latex = generate_dual_meet_tables()

# Save to file
output_file = '/home/ben/Desktop/Projects/media_guide/latex/sections/gen_dual.tex'
with open(output_file, 'w') as f:
    f.write(dual_latex)

print(f"Generated dual meet LaTeX saved to 'gen_dual.tex'")
print(f"Total length: {len(dual_latex)} characters")
print(f"Number of lines: {dual_latex.count(chr(10))}")

# Show preview
print("\nPreview (first 20 lines):")
lines = dual_latex.split('\n')
for i, line in enumerate(lines[:20]):
    print(f"{i+1:2d}: {line}")
if len(lines) > 20:
    print(f"... and {len(lines) - 20} more lines")


Generating dual meet tables...
Found 1016 records for dual meets
Sheets: ['CMS at UCSD', 'CMS at Cal Baptist Distance Meet', 'CMS at PP']

Processing CMS at UCSD...


KeyError: 'TIME'

In [4]:
# Fixed version: Generate dual meet tables using teamrecordtable macro
def generate_dual_meet_tables_fixed():
    """Generate LaTeX for dual meet sheets using teamrecordtable macro in 2-column layout"""
    
    # Define the dual meet sheets we want to process
    dual_meet_sheets = [
        'CMS at UCSD',
        'CMS at Cal Baptist Distance Meet', 
        'CMS at PP'
    ]
    
    # Filter data for these sheets only
    dual_data = df[df['SHEET'].isin(dual_meet_sheets)]
    
    print(f"Found {len(dual_data)} records for dual meets")
    print(f"Sheets: {dual_meet_sheets}")
    print(f"Available columns: {list(dual_data.columns)}")
    
    latex_parts = []
    
    # Add header
    latex_parts.append("% ===== DUAL MEET TABLES =====")
    latex_parts.append("% Generated using teamrecordtable macro")
    latex_parts.append("")
    
    # Helper function to get column value safely (handles spaces in column names)
    def get_column_value(row, column_name):
        """Get column value with fallback to similar column names"""
        # First try exact match
        if column_name in row.index:
            return str(row[column_name]) if pd.notna(row[column_name]) else ""
        
        # Try variations with spaces (common issue with CSV imports)
        variations = [f" {column_name}", f"{column_name} ", f" {column_name} "]
        for var in variations:
            if var in row.index:
                return str(row[var]) if pd.notna(row[var]) else ""
        
        return ""
    
    # Helper function to sort by time
    def sort_by_time(data, time_col_name):
        """Sort data by time column"""
        # Find the actual time column name
        time_col = None
        if time_col_name in data.columns:
            time_col = time_col_name
        else:
            # Try variations with spaces
            variations = [f" {time_col_name}", f"{time_col_name} ", f" {time_col_name} "]
            for var in variations:
                if var in data.columns:
                    time_col = var
                    break
        
        if time_col is None:
            return data  # No time column found, return unsorted
        
        data = data.copy()
        data['time_seconds'] = data[time_col].apply(
            lambda x: float('inf') if pd.isna(x) else 
            (float(str(x).split(':')[0]) * 60 + float(str(x).split(':')[1]) if ':' in str(x) else float(str(x))) if str(x).replace('.', '').replace(':', '').replace('-', '').isdigit() else float('inf')
        )
        data = data.sort_values('time_seconds').drop('time_seconds', axis=1)
        return data.reset_index(drop=True)
    
    # Process each sheet
    for sheet in dual_meet_sheets:
        sheet_data = dual_data[dual_data['SHEET'] == sheet]
        if len(sheet_data) == 0:
            continue
            
        print(f"\nProcessing {sheet}...")
        
        # Escape sheet name for LaTeX
        escaped_sheet_name = sheet.replace('&', '\\&')
        latex_parts.append(f"\\subsection{{{escaped_sheet_name}}}")
        latex_parts.append("")
        
        # Get available sexes for this sheet
        available_sexes = sheet_data['SEX'].unique()
        
        for sex in ['Athena', 'Stag', 'Women', 'Men']:
            if sex not in available_sexes:
                continue
                
            sex_data = sheet_data[sheet_data['SEX'] == sex]
            
            # Escape sex name for LaTeX
            escaped_sex_name = sex.replace('&', '\\&')
            latex_parts.append(f"\\subsubsection{{{escaped_sex_name}}}")
            latex_parts.append("")
            
            # Group by event in order
            events_data = []
            for event in event_order:
                event_data = sex_data[sex_data['EVENT'] == event]
                if len(event_data) > 0:
                    # Sort by time if TIME column exists
                    event_data = sort_by_time(event_data, 'TIME')
                    events_data.append((event, event_data))
            
            # Group events in pairs for 2-column layout
            for i in range(0, len(events_data), 2):
                latex_parts.append("\\begin{table}[H]")
                latex_parts.append("\\centering")
                
                # First event
                if i < len(events_data):
                    event_name, event_data = events_data[i]
                    
                    # Generate table rows for teamrecordtable macro
                    rows = []
                    for _, row in event_data.iterrows():
                        time_val = get_column_value(row, 'TIME')
                        name_val = get_column_value(row, 'NAME')
                        year_val = get_column_value(row, 'YEAR')
                        
                        # Escape special LaTeX characters
                        time_val = time_val.replace('&', '\\&').replace('%', '\\%').replace('$', '\\$')
                        name_val = name_val.replace('&', '\\&').replace('%', '\\%').replace('$', '\\$')
                        
                        rows.append(f"    {time_val} & {name_val} & {year_val} \\\\")
                    
                    table_content = "\n".join(rows)
                    
                    latex_parts.append("\\begin{minipage}[t]{0.48\\textwidth}")
                    latex_parts.append(f"\\teamrecordtable{{{event_name}}}{{")
                    latex_parts.append(table_content)
                    latex_parts.append("}")
                    latex_parts.append("\\end{minipage}\\hfill")
                
                # Second event (if available)
                if i + 1 < len(events_data):
                    event_name, event_data = events_data[i + 1]
                    
                    # Generate table rows for teamrecordtable macro
                    rows = []
                    for _, row in event_data.iterrows():
                        time_val = get_column_value(row, 'TIME')
                        name_val = get_column_value(row, 'NAME')
                        year_val = get_column_value(row, 'YEAR')
                        
                        # Escape special LaTeX characters
                        time_val = time_val.replace('&', '\\&').replace('%', '\\%').replace('$', '\\$')
                        name_val = name_val.replace('&', '\\&').replace('%', '\\%').replace('$', '\\$')
                        
                        rows.append(f"    {time_val} & {name_val} & {year_val} \\\\")
                    
                    table_content = "\n".join(rows)
                    
                    latex_parts.append("\\begin{minipage}[t]{0.48\\textwidth}")
                    latex_parts.append(f"\\teamrecordtable{{{event_name}}}{{")
                    latex_parts.append(table_content)
                    latex_parts.append("}")
                    latex_parts.append("\\end{minipage}")
                else:
                    # If only one event, close the minipage properly
                    latex_parts[-1] = "\\end{minipage}"  # Remove \hfill
                
                latex_parts.append("\\end{table}")
                latex_parts.append("")
    
    return "\n".join(latex_parts)

# Generate the dual meet tables with fixed code
print("Generating dual meet tables with fixed column handling...")
dual_latex = generate_dual_meet_tables_fixed()

# Save to file
output_file = '/home/ben/Desktop/Projects/media_guide/latex/sections/gen_dual.tex'
with open(output_file, 'w') as f:
    f.write(dual_latex)

print(f"Generated dual meet LaTeX saved to 'gen_dual.tex'")
print(f"Total length: {len(dual_latex)} characters")
print(f"Number of lines: {dual_latex.count(chr(10))}")

# Show preview
print("\nPreview (first 20 lines):")
lines = dual_latex.split('\n')
for i, line in enumerate(lines[:20]):
    print(f"{i+1:2d}: {line}")
if len(lines) > 20:
    print(f"... and {len(lines) - 20} more lines")


Generating dual meet tables with fixed column handling...
Found 1016 records for dual meets
Sheets: ['CMS at UCSD', 'CMS at Cal Baptist Distance Meet', 'CMS at PP']
Available columns: ['SHEET', 'SEX', 'EVENT', ' TIME', 'NAME', 'YEAR', 'TEAM', 'RANK', 'SITE', 'MEET', 'CONTEXT']

Processing CMS at UCSD...

Processing CMS at Cal Baptist Distance Meet...

Processing CMS at PP...
Generated dual meet LaTeX saved to 'gen_dual.tex'
Total length: 51286 characters
Number of lines: 1632

Preview (first 20 lines):
 1: % ===== DUAL MEET TABLES =====
 2: % Generated using teamrecordtable macro
 3: 
 4: \subsection{CMS at UCSD}
 5: 
 6: \subsubsection{Athena}
 7: 
 8: \begin{table}[H]
 9: \centering
10: \begin{minipage}[t]{0.48\textwidth}
11: \teamrecordtable{50 FREE}{
12:     24.45 & Kelly Ngo & 2014 \\
13:     24.50 & Lorea Gwo & 2014 \\
14:     24.53 & Ava Sealander & 2019 \\
15:     24.58 & Kelly Ngo & 2016 \\
16:     24.64 & Annette Chang & 2023 \\
17:     24.65 & Kelly Ngo & 2015 \\
18:     24.